In [7]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [39]:
class GlaucomaIdentifier:
    def __init__(self, img, discDiameter=1, cupDiameter=0):
        self.image = img
        self.discDiameter = discDiameter
        self.cupDiameter = cupDiameter
        
    def getYComponent(self,pixel):
        #Converts RGB pixel to YCrcb and gets the Y component
        #@param pixel
        #@return Y component of the pixel
        blue, green, red = pixel
        #print("pixel"+str(pixel))
        yComponent = ( (red*0.299) + (green*0.587) + (blue*0.144) )
        #print(yComponent)
        return yComponent
    
    def getVCDR(self):
        #Calculates Vertical Cup to Disc ratio (VCDR)
        #@return VCDR
        vcdr = float(self.cupDiameter) / float(self.discDiameter)
        vcdr = np.around(vcdr * 100.00)/100.00
        return vcdr
    
    def getCupImage(self):
        #Segments cup region, changing all its pixels to black and calculates cup diameter
        #@return Image with cup region segmented
        cupImage = GlaucomaIdentifier(np.copy(self.image))
        minCup = 99999
        maxCup = -1
        height, width, _ = cupImage.image.shape

        for j in range(height):
            for i in range(width):
                if(cupImage.getYComponent(cupImage.image[j][i]) > 200):
                    #print("yeah")
                    cupImage.image[j][i] = (0,0,0)
                    if(j < minCup):
                        minCup = j
                    if(j > maxCup):
                        maxCup = j
                    #print(cupImage.image[j][i])
        #cv2.imshow("cupImage",cupImage.image)           
        print("Max cup = " + str(maxCup) +", Min Cup = "+ str(minCup))
        self.cupDiameter = maxCup - minCup
        return cupImage.image

    def getDiscImage(self):
        #Segments disc region, changing all its pixels to white and the others to black, and calculates cup diameter
        #@return Image with disc region segmented
        discImage = GlaucomaIdentifier(np.copy(self.image))
        minDisc = 99999
        maxDisc = -1
        height, width, _ = discImage.image.shape

        for j in range(height):
            for i in range(width):
                if(discImage.image[j][i][2] >= 230):
                    #print("yeah")
                    discImage.image[j][i] = (255,255,255)
                    if(j < minDisc):
                        minDisc = j
                    if(j > maxDisc):
                        maxDisc = j
                    #print(cupImage.image[j][i])
                else:
                    discImage.image[j][i] = (0,0,0)
        #cv2.imshow("cupImage",cupImage.image)           
        print("Max Disc = " + str(maxDisc) +", Min Disc = "+ str(minDisc))
        self.cupDiameter = maxDisc - minDisc
        return discImage.image

    def getCupToDiscImage(self):
        cupDiscImage = np.copy(self.getDiscImage())
        cupImage = np.copy(self.getCupImage())
        height, width,_ = cupDiscImage.shape

        for j in range(height):
            for i in range(width):
                if(np.any(cupDiscImage[j][i] ==  255)):
                    if(np.any(cupImage[j][i] ==  0)):
                        cupDiscImage[j][i] = (0,0,0)

        return cupDiscImage


In [40]:
sample = cv2.imread("01_g.jpg")
resized_img = cv2.resize(sample,(480,360))
im = GlaucomaIdentifier(np.copy(resized_img))
#print(im.getYComponent(1,1,1))
#print(im.getVCDR())

print(sample.shape)
print(im.image.shape)

cupImage = im.getCupToDiscImage()
print("Cup Diameter = "+str(im.cupDiameter))
pixel = sample[0][0]
print("sampleAntes"+str(resized_img[0][0]))
print("pixelAntes"+str(pixel))
print(pixel)
pixel = (0,0,0)
resized_img[0][0] = (0,0,0)
print(pixel)
print("sampleDepois"+str(resized_img[0][0]))
if( np.any(resized_img[0][0] == 0) ):
    print("daora")
print("pixelDepois"+str(pixel))
cv2.imshow("original",resized_img)
cv2.imshow("cupImage",cupImage)
#print(cupImage == sample)
# waits for user to press any key 
# (this is necessary to avoid Python kernel form crashing) 
cv2.waitKey(0)
# closing all open windows 
cv2.destroyAllWindows()

(2336, 3504, 3)
(360, 480, 3)
Max Disc = 359, Min Disc = 58
Max cup = -1, Min Cup = 99999
Cup Diameter = -100000
sampleAntes[0 1 1]
pixelAntes[0 2 0]
[0 2 0]
(0, 0, 0)
sampleDepois[0 0 0]
daora
pixelDepois(0, 0, 0)
